In [9]:
import os
import sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
MODDED_SAVE_DIR = os.getenv("MODDED_SAVE_DIR")
TARGET_GAME = "F1Manager23"
TARGET_SAVE = "save5"

sqlite_db_path = f"{MODDED_SAVE_DIR}\\{TARGET_GAME}\\save_db\\{TARGET_SAVE}\\main.db"

In [10]:
CLASSES_DIR = os.getenv("SRC_DIR")
sys.path.append(CLASSES_DIR)
from Model.DateConverter import DateConverter

In [11]:
import pandas as pd
import sqlite3

con = sqlite3.connect(sqlite_db_path)
teams = pd.read_sql_query(
    sql="select * from Teams",
    con=con
)
teams = teams[['Formula', 'PredictedRanking', 'TeamID', 'TeamName', 'TeamNameLocKey', 'Founded', 'SeasonsEnteredBeforeGameStart', 'FirstYearWithCurrentBranding']]
teams['Founded'] = teams['Founded'].apply(lambda x: x if x != 0 else None )
teams.head()

,Formula,PredictedRanking,TeamID,TeamName,TeamNameLocKey,Founded,SeasonsEnteredBeforeGameStart,FirstYearWithCurrentBranding
0,1,6,1,Ferrari,[TeamName_F1_Ferrari],1950.0,73.0,1950
1,1,3,2,McLaren,[TeamName_F1_McLaren],1966.0,57.0,1966
2,1,2,3,Red Bull Racing,[TeamName_F1_RedBullRacing],2005.0,18.0,2005
3,1,1,4,Mercedes AMG Petronas F1,[TeamName_F1_MercedesAMGPetronasF1],2010.0,13.0,2010
4,1,5,5,Alpine,[TeamName_F1_Alpine],1986.0,26.0,2021


In [12]:
transactions = pd.read_sql_query(
    sql="select * from Finance_Transactions where Reference > 2021",
    con=con
)
transactions['Day'] = transactions['Day'].apply(lambda x: DateConverter(x).get_date())
transactions.head()

,TeamID,Day,Value,TransactionType,Reference,AffectsCostCap
0,1,2023-01-01,-3638411,11,2023,0
1,2,2023-01-01,-1441026,11,2023,0
2,3,2023-01-01,-5624352,11,2023,0
3,4,2023-01-01,-3421454,11,2023,0
4,5,2023-01-01,-1518908,11,2023,0


In [13]:
balance = pd.read_sql_query(
    sql="select * from Finance_TeamBalance",
    con=con
)
balance.head()

,TeamID,Balance
0,1,15292349
1,2,16303588
2,3,17486600
3,4,15560155
4,5,27390810


In [14]:
with pd.ExcelWriter('TeamsFinancing.xlsx') as writer:
    teams.to_excel(writer, sheet_name='Teams', index=False)
    transactions.to_excel(writer, sheet_name='Transactions', index=False)
    balance.to_excel(writer, sheet_name='Balance', index=False)